# Color detection in realtime

In [74]:
import pandas as pd
import cv2


In [75]:
img_path = 'data/colorbuilding.jpg'
csv_path = 'data/colors.csv'

## Exploratory Data Analysis

In [76]:
#creating a dataframe to fetch the data
df = pd.read_csv(csv_path)
df.head()

,air_force_blue_raf,Air Force Blue (Raf),#5d8aa8,93,138,168
0,air_force_blue_usaf,Air Force Blue (Usaf),#00308f,0,48,143
1,air_superiority_blue,Air Superiority Blue,#72a0c1,114,160,193
2,alabama_crimson,Alabama Crimson,#a32638,163,38,56
3,alice_blue,Alice Blue,#f0f8ff,240,248,255
4,alizarin_crimson,Alizarin Crimson,#e32636,227,38,54


Since there is no headers, it shows 6 rows of data.

In [77]:
#number of rows
rows = len(df.axes[0])
print(rows)

864


In [78]:
#Adding a header
index = ['color', 'color_name', 'hex', 'R', 'G', 'B']
df = pd.read_csv(csv_path, names = index, header=None)
df

,color,color_name,hex,R,G,B
0,air_force_blue_raf,Air Force Blue (Raf),#5d8aa8,93,138,168
1,air_force_blue_usaf,Air Force Blue (Usaf),#00308f,0,48,143
2,air_superiority_blue,Air Superiority Blue,#72a0c1,114,160,193
3,alabama_crimson,Alabama Crimson,#a32638,163,38,56
4,alice_blue,Alice Blue,#f0f8ff,240,248,255
...,...,...,...,...,...,...
860,yellow_orange,Yellow Orange,#ffae42,255,174,66
861,yellow_process,Yellow (Process),#ffef00,255,239,0
862,yellow_ryb,Yellow (Ryb),#fefe33,254,254,51
863,zaffre,Zaffre,#0014a8,0,20,168


In [79]:
#fetching any single row using index
print(df.loc[1])

color           air_force_blue_usaf
color_name    Air Force Blue (Usaf)
hex                         #00308f
R                                 0
G                                48
B                               143
Name: 1, dtype: object


In [80]:
#slicing the rgb values
print(df.loc[1,'B'])

143


## Reading Image

In [81]:
#Reading an image
img = cv2.imread(img_path)
resized = cv2.resize(img,(800,600))
# cv2.imshow("Image", resized)
# cv2.waitKey(0)

In [82]:
#Initializing variables
clicked = False
r = g = b = xpos = ypos = 0


### Fetching the pixel value from the image using mouse click

In [83]:
#Get color names from the csv file
def get_color_name(R, G, B):
    min = 1000
    for i in range(len(df)):
        #To match the color, we take absolute difference of each color channel from image and df.
        d = abs(R - int(df.loc[i,'R'])) + abs(G - int(df.loc[i,'G'])) + abs(B-int(df.loc[i,'B']))
        if d <= min:
            min = d
            cname = df.loc[i, 'color_name']
    return cname

In [84]:
def draw_function(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDBLCLK:
        global clicked, r, g, b, xpos, ypos
        clicked = True
        xpos = x
        ypos = y
        #print(x,y)
        b, g, r = resized[y,x]
        #print(b,g,r)
        b = int(b)
        g = int(g)
        r = int(r)
        #print (r, g, b)


In [85]:

# cv2.imshow("Image", resized)
# cv2.setMouseCallback("Image", draw_function)
# cv2.waitKey(0)

In [86]:
#Sanity check
#print(get_color_name(0,0,0))

## Displaying Output

In [87]:
cv2.namedWindow("Image")
cv2.setMouseCallback("Image", draw_function)
while True:
    cv2.imshow("Image", resized)
    if clicked:
        #print("Clicked")
        #Creating a rectangle to show the color clicked on.
        cv2.rectangle(resized, (20,20), (600,60), (b,g,r), -1)
        text = get_color_name(r, g, b) + '  R=' + str(r) +' G='+ str(g) +' B=' + str(b)
        cv2.putText(resized, text, (50,50), 2, 0.8, (255,255,255), 2, cv2.LINE_AA)
        #if the color is too light, then writing text in white does not make sense, so
        if r+g+b >= 600:
            cv2.putText(resized, text, (50,50), 2, 0.8, (0,0,0), 2, cv2.LINE_AA)
    if cv2.waitKey(20) & 0xFF == 27:
        break
cv2.destroyAllWindows()
